# Data cleaning
Data from the ENTSO-E API and the weather data must be cleaned, combined and then saved to a new file.

In [2]:
import pandas as pd
import numpy as np

path = '/Users/jakagodec/Desktop/ist/es/project3/EnergyPrices/'

## Transformations
Below are functions to load and transform data from each raw file. Mostly they just set a `DatetimeIndex` and rename some columns. The biggest exception is the unavailability data which has to be transformed.

In [142]:
def get_prices(code):
    country_path = path + 'data/' + code + '_raw/' + code
    df_prices = pd.read_csv(country_path + '_prices.csv')
    df_prices['Time'] = pd.to_datetime(pd.to_datetime(df_prices['Unnamed: 0']), utc=True)
    
    return df_prices.set_index('Time').drop('Unnamed: 0', axis=1).rename({'0': 'DayAheadPrice'}, axis=1)

def get_generation(code):
    country_path = path + 'data/' + code + '_raw/' + code
    df_generation = pd.read_csv(country_path + '_generation.csv')
    df_generation['Time'] = pd.to_datetime(pd.to_datetime(df_generation['Unnamed: 0']), utc=True)
    map_generation = {
        'Biomass': 'GenBiomass', 
        'Fossil Brown coal/Lignite': 'GenFossilBrownCoal', 
        'Fossil Gas': 'GenFossilGas', 
        'Fossil Oil': 'GenFossilOil',
        'Fossil Hard coal': 'GenFossilHardCoal',
        'Hydro Pumped Storage': 'GenHydroPumped', 
        'Hydro Pumped Storage.1': 'GenHydroPumpedOther',
        'Hydro Run-of-river and poundage': 'GenHydroRiver',
        'Hydro Water Reservoir': 'GenHydroReservoir',
        'Nuclear': 'GenNuclear',
        'Geothermal': 'GenGeothermal',
        'Other renewable': 'GenOtherRenewable',
        'Solar': 'GenSolar', 
        'Waste': 'GenWaste', 
        'Wind Onshore': 'GenWind',
        'Other': 'GenOther'
        }
    
    return df_generation.set_index('Time').drop('Unnamed: 0', axis=1).rename(map_generation, axis=1)


def get_import(code):
    country_path = path + 'data/' + code + '_raw/' + code
    df_import = pd.read_csv(country_path + '_import.csv')
    df_import['Time'] = pd.to_datetime(pd.to_datetime(df_import['Unnamed: 0']), utc=True)
    df_import = df_import.set_index('Time').drop('Unnamed: 0', axis=1).fillna(0).resample('H').sum()
    map_import = {col: 'Import' + col for col in df_import.columns}
    df_import.rename(map_import, axis=1, inplace=True)
    
    return df_import

def get_loads(code):
    country_path = path + 'data/' + code + '_raw/' + code
    df_load = pd.read_csv(country_path + '_load.csv')
    df_load['Time'] = pd.to_datetime(pd.to_datetime(df_load['Unnamed: 0']), utc=True)
    map_load = {'Forecasted Load': 'LoadForecast', 'Actual Load': 'LoadActual'}
    
    return df_load.set_index('Time').drop('Unnamed: 0', axis=1).rename(map_load, axis=1)


def get_unavailability(code):
    global_start = pd.to_datetime(pd.Timestamp('20220101', tz='CET'), utc=True)
    global_end = pd.to_datetime(pd.Timestamp('20230301', tz='CET'), utc=True)
    global_index = pd.date_range(global_start, global_end, freq='h')

    sum = pd.Series(np.zeros(global_index.shape[0]), index=global_index)

    df = pd.read_csv(path + 'data/' + code + '_raw/' + code + '_unavailability.csv')#[['start', 'end', 'nominal_power']]
    for row in df.index:
        t_start = pd.to_datetime(pd.to_datetime(df.loc[row, 'start']), utc=True)
        t_end = pd.to_datetime(pd.to_datetime(df.loc[row, 'end']), utc=True)
        i = pd.date_range(t_start, t_end, freq='h')
        val = df.loc[row, 'nominal_power'] * np.ones(i.shape[0])
        sum = sum.add(pd.Series(val, index=i), fill_value=0)

    return pd.DataFrame(sum.reindex(global_index, fill_value=0), columns=['PowerUnavailable'])
    

In [123]:
def get_weather(code):
    country_path = path + 'data/' + code + '_raw/' + code

    df = pd.read_excel(country_path + '_weather.xlsx')
    map = {
        'latitude': 'Time',
        'longitude': 'Temperature',
        'elevation': 'Precipitation',
        'utc_offset_seconds': 'SolarRad'
    }
    df = df.loc[3:].drop(['timezone', 'timezone_abbreviation'], axis=1).rename(map, axis=1)
    return df.set_index(pd.to_datetime(pd.to_datetime(df['Time']), utc=True)).drop('Time', axis=1).iloc[:-24]
    

## Combining data
Function below loads and joins all files for given country.

In [132]:
def clean_data(code):
    df_prices = get_prices(code)
    df_generation = get_generation(code)
    df_import = get_import(code)
    df_load = get_loads(code)
    df_weather = get_weather(code)

    # join and return
    df_main = df_prices.merge(
        df_generation, 
        left_index=True, 
        right_index=True
    ).merge(
            df_import, 
            left_index=True, 
            right_index=True
        ).merge(
                df_load, 
                left_index=True, 
                right_index=True
            ).merge(
                    df_weather,
                    left_index=True,
                    right_index=True
                )
    
    if code.lower() != 'hr':
        df_unav = get_unavailability(code)
    
        return df_main.merge(df_unav, left_index=True, right_index=True)
    
    return df_main

## Saving data
Data for a single country (`pandas.DataFrame`) is saved as a pickle (`.pkl`) file as it can then be loaded as a `pandas.DataFrame` immediately.

In [145]:
import pickle

In [146]:
codes = ['SI', 'HR', 'RS', 'RO', 'GR', 'BG']

for c in codes:
    df = clean_data(c)
    fname = path + 'data/clean/' + c + '_clean.pkl'
    with open(fname, 'wb') as f:
        pickle.dump(df, f)